In [6]:
from requests import post, get
from requests.models import Response
from mlserver.codecs.string import StringRequestCodec
from mlserver.types import InferenceResponse, InferenceRequest
import json

In [7]:
def create_request_to_mlserver_model (data: dict, model_name: str) -> dict:
    j_input = json.dumps(data)
    dict_inference_request = {
        "inputs": [
            {
              "name": model_name,
              "shape": [len(j_input)],
              "datatype": "BYTES",
              "data": j_input,
            }
          ]
    }
    print(f"Type dict_inference_request {type(dict_inference_request)}")
    return dict_inference_request

In [9]:
d = {"a": 10}
ip = "158.160.52.53"
model_name = "work_model1"
host = f"http://{ip}:8000"
test_inf_resp = create_request_to_mlserver_model(d, model_name)
test_inf_resp

Type dict_inference_request <class 'dict'>


{'inputs': [{'name': 'work_model1',
   'shape': [9],
   'datatype': 'BYTES',
   'data': '{"a": 10}'}]}

In [10]:
def send_request_to_mlserver_model(inference_request: dict, host: str, model_name: str) -> Response:
    endpoint = f"{host}/v2/models/{model_name}/infer"
    headers = {"Content-Type": "application/json"}
    post_response = post(endpoint, json=inference_request, headers=headers)
    print(f"Type post_response {type(post_response)}")
    return post_response

In [11]:
model_answer = send_request_to_mlserver_model(test_inf_resp, host, model_name)
model_answer

Type post_response <class 'requests.models.Response'>


<Response [200]>

In [12]:
def decode_answer(post_response: Response) -> dict:
    print(f"Response (от POST запроса) для decode_answer: {Response}")
    inf_resp = InferenceResponse.parse_raw(post_response.text)
    print(f"inf_resp в decode_answer: {inf_resp}")
    raw_json = StringRequestCodec.decode_response(inf_resp)
    print(f"raw_json в decode_answer: {raw_json}")
    res = json.loads(raw_json[0])
    print(f"raw_json[0] = {raw_json[0]}")
    return res

In [13]:
output = decode_answer(model_answer)
output

Response (от POST запроса) для decode_answer: <class 'requests.models.Response'>
inf_resp в decode_answer: model_name='work_model1' model_version=None id='a0b56624-4560-4323-9c1d-31d25bc3873e' parameters=Parameters(content_type='dict', headers=None) outputs=[ResponseOutput(name='output', shape=[1], datatype='BYTES', parameters=Parameters(content_type='np', headers=None), data=TensorData(__root__=['{"result": 20}']))]
raw_json в decode_answer: ['{"result": 20}']
raw_json[0] = {"result": 20}


{'result': 20}